### 계약전력합계 : 해당 건물이 계약한 최대 전력량
### 전력수요합계 : 주어진 시간 동안 소비된 총 전력량
### 전력부하량 평균 : 주어진 시간 동안의 평균 전력 부하량

In [1]:
import pandas as pd

In [16]:
train = pd.read_csv("C:/Users/삼성/OneDrive - 계명대학교/바탕 화면/날씨빅콘/날씨 데이터셋/파생변수_train.csv")
test = pd.read_csv("C:/Users/삼성/OneDrive - 계명대학교/바탕 화면/날씨빅콘/real2_ele_te.csv")

In [3]:
train.columns

Index(['Unnamed: 0', '격자넘버', '날짜', '시간', '공동주택수', '지점번호', '계약전력합계', '전력수요합계',
       '전력부하량평균', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말', '전력기상지수',
       '계절', '년', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '상대습도_계절별평균',
       '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균'],
      dtype='object')

In [17]:
test.columns

Index(['Unnamed: 0', 'electric_test.num', 'electric_test.tm',
       'electric_test.hh24', 'electric_test.stn', 'electric_test.nph_ta',
       'electric_test.nph_hm', 'electric_test.nph_ws_10m',
       'electric_test.nph_rn_60m', 'electric_test.nph_ta_chi',
       'electric_test.weekday', 'electric_test.week_name'],
      dtype='object')

In [18]:
test.rename(columns={'electric_test.num': '격자넘버'}, inplace=True)
test.rename(columns={'electric_test.tm': '날짜'}, inplace=True)
test.rename(columns={'electric_test.hh24': '시간'}, inplace=True)
test.rename(columns={'electric_test.stn': '지점번호'}, inplace=True)
test.rename(columns={'electric_test.nph_ta': '기온'}, inplace=True)
test.rename(columns={'electric_test.nph_hm': '상대습도'}, inplace=True)
test.rename(columns={'electric_test.nph_ws_10m': '풍속'}, inplace=True)
test.rename(columns={'electric_test.nph_rn_60m': '강수량'}, inplace=True)
test.rename(columns={'electric_test.nph_ta_chi': '체감온도'}, inplace=True)
test.rename(columns={'electric_test.weekday': '요일'}, inplace=True)
test.rename(columns={'electric_test.week_name': '주중주말'}, inplace=True)

In [19]:
# 파생변수생성
# CDH, HDH 생성

# 기준 온도 설정 (일반적으로 18°C)
base_temp = 18.0

# CDH와 HDH 계산 함수
def calculate_cdh(temp, base_temp):
    return max(0, temp - base_temp)

def calculate_hdh(temp, base_temp):
    return max(0, base_temp - temp)

# test 셋
test['CDH'] = test.apply(lambda x: calculate_cdh(x['기온'], base_temp), axis=1)
test['HDH'] = test.apply(lambda x: calculate_hdh(x['기온'], base_temp), axis=1)

# 불쾌지수 컬럼 추가
def calculate_discomfort_index(temperature, humidity):
    return 1.8 * temperature - 0.55 * (1 - humidity / 100) * (1.8 * temperature - 26) + 32

# test 셋
test['불쾌지수'] = test.apply(lambda x: calculate_discomfort_index(x['기온'], x['상대습도']), axis=1)

In [20]:
test = test.sort_values(by='날짜')
test['날짜'] = pd.to_datetime(test['날짜'])

def assign_season(date):
    if (date.month == 2) or (date.month == 3) or (date.month == 4):
        return '봄'
    elif (date.month == 5) or (date.month == 6) or (date.month == 7):
        return '여름'
    elif (date.month == 8) or (date.month == 9) or (date.month == 10):
        return '가을'
    else:
        return '겨울'

# '계절' 컬럼 생성
test['계절'] = test['날짜'].apply(assign_season)

def assign_date(df, date_column):
    df['년'] = df[date_column].dt.year
    df['월'] = df[date_column].dt.month
    df['일'] = df[date_column].dt.day
    df['시간'] = df[date_column].dt.hour
    return df

test = assign_date(test, '날짜')
test.head(3)

,Unnamed: 0,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,CDH,HDH,불쾌지수,계절,년,월,일
0,1,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1,0.0,15.0,40.957620,겨울,2023,1,1
2627998,2627999,18574,2023-01-01 01:00:00,1,876,2.1,48.3,0.5,0.0,2.1,6,1,0.0,15.9,42.098257,겨울,2023,1,1
201480,201481,18828,2023-01-01 01:00:00,1,511,0.2,54.3,1.9,0.0,-0.3,6,1,0.0,17.8,38.804614,겨울,2023,1,1


In [21]:
# 계절별 평균 계산
seasonal_averages = test.groupby('계절').agg({
    '기온': 'mean',
    '상대습도': 'mean',
    '체감온도': 'mean',
    '풍속': 'mean',
    '강수량': 'mean'
}).reset_index()

test = test.merge(seasonal_averages, on='계절', suffixes=('', '_계절별평균'))

In [22]:
test.columns

Index(['Unnamed: 0', '격자넘버', '날짜', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량',
       '체감온도', '요일', '주중주말', 'CDH', 'HDH', '불쾌지수', '계절', '년', '월', '일',
       '기온_계절별평균', '상대습도_계절별평균', '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균'],
      dtype='object')

### 계절별 모델 앙상블

In [38]:
# 필요한 열만 뽑기
train_drop = train.drop(columns = ['Unnamed: 0','날짜','공동주택수','계약전력합계','전력부하량평균',
                                   '년','체감온도_계절별평균','상대습도_계절별평균','전력수요합계'])

test_drop = test.drop(columns = ['Unnamed: 0','상대습도_계절별평균','체감온도_계절별평균','날짜','년'])

In [39]:
print(train_drop.columns)
print(test_drop.columns)

Index(['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
       '전력기상지수', '계절', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '풍속_계절별평균',
       '강수량_계절별평균'],
      dtype='object')
Index(['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
       'CDH', 'HDH', '불쾌지수', '계절', '월', '일', '기온_계절별평균', '풍속_계절별평균',
       '강수량_계절별평균'],
      dtype='object')


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 모델 저장을 위한 딕셔너리 생성
models = {}

# 계절 변수를 원핫인코딩
train_drop = pd.get_dummies(train_drop, columns=['계절'])

# 원핫 인코딩 후 생성된 열 이름 확인
print(train_drop.columns)

Index(['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
       '전력기상지수', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '풍속_계절별평균',
       '강수량_계절별평균', '계절_가을', '계절_겨울', '계절_봄', '계절_여름'],
      dtype='object')


In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm  # tqdm 라이브러리 임포트
import lightgbm as lgb

# 모델 저장을 위한 딕셔너리 생성
models = {}

# 계절별로 데이터 분할 및 모델 학습
seasons = ['계절_가을', '계절_겨울', '계절_봄', '계절_여름']
for season in tqdm(seasons, desc="Season Progress"):
    season_data = train_drop[train_drop[season] == 1]
    
    X = season_data.drop(columns=['전력기상지수'])
    y = season_data['전력기상지수']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)
    
    models[season] = model
    
    y_pred = model.predict(X_test)
    print(f'계절 {season} - MAE:', mean_absolute_error(y_test, y_pred))
    print(f'계절 {season} - RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print(f'계절 {season} - r2:', r2_score(y_test, y_pred))


Season Progress:   0%|          | 0/4 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.125254 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2287
[LightGBM] [Info] Number of data points in the train set: 1529649, number of used features: 15
[LightGBM] [Info] Start training from score 103.805943


Season Progress:  25%|██▌       | 1/4 [00:27<01:23, 27.95s/it]

계절 계절_가을 - MAE: 4.595052171075787
계절 계절_가을 - RMSE: 6.530686485587127
계절 계절_가을 - r2: 0.9530239274314444
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.183889 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2067
[LightGBM] [Info] Number of data points in the train set: 1529701, number of used features: 15
[LightGBM] [Info] Start training from score 99.580686


Season Progress:  50%|█████     | 2/4 [00:49<00:48, 24.39s/it]

계절 계절_겨울 - MAE: 3.084992419474737
계절 계절_겨울 - RMSE: 4.0644827840651825
계절 계절_겨울 - r2: 0.9590043839453648
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084194 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2137
[LightGBM] [Info] Number of data points in the train set: 1485578, number of used features: 15
[LightGBM] [Info] Start training from score 95.453931


Season Progress:  75%|███████▌  | 3/4 [01:10<00:22, 22.75s/it]

계절 계절_봄 - MAE: 2.8964500690856134
계절 계절_봄 - RMSE: 3.7928242147015405
계절 계절_봄 - r2: 0.960439446103591
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.168183 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2187
[LightGBM] [Info] Number of data points in the train set: 1529615, number of used features: 15
[LightGBM] [Info] Start training from score 101.017813


Season Progress: 100%|██████████| 4/4 [01:32<00:00, 23.18s/it]

계절 계절_여름 - MAE: 4.393342639344027
계절 계절_여름 - RMSE: 6.26333592867341
계절 계절_여름 - r2: 0.9514075626895879


In [31]:
test.columns

Index(['Unnamed: 0', 'electric_test.num', 'electric_test.tm',
       'electric_test.hh24', 'electric_test.stn', 'electric_test.nph_ta',
       'electric_test.nph_hm', 'electric_test.nph_ws_10m',
       'electric_test.nph_rn_60m', 'electric_test.nph_ta_chi',
       'electric_test.weekday', 'electric_test.week_name'],
      dtype='object')

In [45]:
test_drop.head()

,격자넘버,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,CDH,HDH,불쾌지수,계절,월,일,기온_계절별평균,풍속_계절별평균,강수량_계절별평균
0,4816,1,752,3.0,68.6,2.9,0.0,-0.1,6,1,0.0,15.0,40.957620,겨울,1,1,5.729009,2.061995,0.0742
1,18574,1,876,2.1,48.3,0.5,0.0,2.1,6,1,0.0,15.9,42.098257,겨울,1,1,5.729009,2.061995,0.0742
2,18828,1,511,0.2,54.3,1.9,0.0,-0.3,6,1,0.0,17.8,38.804614,겨울,1,1,5.729009,2.061995,0.0742
3,17794,1,436,-2.5,69.6,0.5,0.0,-4.1,6,1,0.0,20.5,32.599600,겨울,1,1,5.729009,2.061995,0.0742
4,19136,1,451,1.1,47.3,1.6,0.0,-2.5,6,1,0.0,16.9,40.942197,겨울,1,1,5.729009,2.061995,0.0742


In [53]:
len(train_drop.columns)

23

In [57]:
test_drop.columns

Index(['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
       'CDH', 'HDH', '불쾌지수', '월', '일', '기온_계절별평균', '풍속_계절별평균', '강수량_계절별평균',
       '계절_가을', '계절_겨울', '계절_봄', '계절_여름'],
      dtype='object')

In [48]:
# 계절 변수를 원핫인코딩
test_drop = pd.get_dummies(test_drop, columns=['계절'])

In [62]:
from tqdm import tqdm

# 새로운 데이터에 대해 예측
def predict_new_data(test_drop, models):
    predictions = []
    # tqdm으로 감싸서 진행 상황을 표시
    for _, row in tqdm(test_drop.iterrows(), desc="Prediction Progress", total=len(test_drop)):
        # 계절 정보 추출
        season_columns = ['계절_가을', '계절_겨울', '계절_봄', '계절_여름']
        season = row[season_columns].idxmax()
        model = models[season]
        
        # 학습할 때 사용한 특징만 추출
        feature_columns = ['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
                           'CDH', 'HDH', '불쾌지수', '월', '일', '기온_계절별평균', '풍속_계절별평균', '강수량_계절별평균',
                           '계절_가을', '계절_겨울', '계절_봄', '계절_여름']
        features = row[feature_columns].values.reshape(1, -1)
        prediction = model.predict(features)
        predictions.append(prediction[0])
    
    return predictions

# 새로운 데이터셋 (test_drop)을 불러와 예측 수행
predictions = predict_new_data(test_drop, models)
print(predictions)


Prediction Progress:   8%|▊         | 237896/2829478 [14:29<2:37:54, 273.53it/s]


KeyboardInterrupt: 

In [12]:
pip install lightgbm

In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [18]:
# 특징 변수와 타겟 변수 분리
X = train_drop.drop(['전력수요합계'], axis=1)  # 타겟 변수는 '기온'이라고 가정
y = train_drop['전력수요합계']

# 원핫 인코딩 및 스케일링
categorical_features = ['계절']
numeric_features = [col for col in X.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 파이프라인 생성
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', lgb.LGBMRegressor())])

model.fit(X_train, y_train)

# 모델 성능 평가
y_pred = model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Validation MAE by Random Search:", mae)
print("Validation MSE by Random Search:", mse)
print("Validation RMSE by Random Search:", rmse)
print("Validation R² by Random Search:", r2)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.444215 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2628
[LightGBM] [Info] Number of data points in the train set: 6074544, number of used features: 23
[LightGBM] [Info] Start training from score 9410.300111
Validation MAE by Random Search: 2089.923055598035
Validation MSE by Random Search: 9423587.924153812
Validation RMSE by Random Search: 3069.7862994276675
Validation R² by Random Search: 0.8761807325638306
